In [2]:
# import dependencies
import pandas as pd
# from geopy.geocoders import Nominatim
import requests
import json
import gmaps
from pandas import json_normalize
from config import API_KEY

In [4]:
# Read in dataset
housing_data = pd.read_csv("Resources/Clean/San_Diego_Housing_Data.csv", index_col=0)
# Reset index to add back zipcodes
housing_data.reset_index(inplace=True)
housing_data

,Geographic Area Name,Households_Total,"Households_Total_Less than $10,000","Households_Total_$10,000 to $14,999","Households_Total_$15,000 to $24,999","Households_Total_$25,000 to $34,999","Households_Total_$35,000 to $49,999","Households_Total_$50,000 to $74,999","Households_Total_$75,000 to $99,999","Households_Total_$100,000 to $149,999",...,"Nonfamily households_Total_$25,000 to $34,999","Nonfamily households_Total_$35,000 to $49,999","Nonfamily households_Total_$50,000 to $74,999","Nonfamily households_Total_$75,000 to $99,999","Nonfamily households_Total_$100,000 to $149,999","Nonfamily households_Total_$150,000 to $199,999","Nonfamily households_Total_$200,000 or more",Nonfamily households_Median income (dollars),Nonfamily households_Mean income (dollars),Nonfamily households_PERCENT ALLOCATED_Nonfamily income in the past 12 months
0,91901,6150.0,2.4,1.0,5.3,10.5,8.0,9.4,16.4,19.8,...,23.4,14.8,13.9,12.6,12.3,3.3,2.0,43523.0,62711.0,22.2
1,91902,6045.0,2.8,0.6,3.1,5.1,7.6,12.1,14.3,23.1,...,13.5,19.1,9.8,13.9,18.6,7.8,7.4,70469.0,111325.0,53.8
2,91906,1282.0,4.6,2.7,7.4,12.6,27.1,11.2,6.4,17.9,...,8.9,41.0,7.0,0.8,0.0,4.3,0.0,35991.0,37850.0,38.0
3,91910,24599.0,4.4,5.4,6.7,9.2,10.4,17.0,13.2,17.3,...,11.2,11.9,18.0,10.9,9.1,2.4,2.9,40825.0,56747.0,44.1
4,91911,25145.0,3.2,5.2,7.7,9.7,11.1,18.7,12.1,19.5,...,16.8,14.1,15.6,8.5,6.3,1.6,0.7,33231.0,45098.0,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,92131,12127.0,2.3,0.7,2.2,2.5,4.9,10.5,12.1,20.4,...,5.3,9.5,21.3,17.7,21.4,6.9,8.6,82864.0,100669.0,24.1
81,92139,10066.0,3.0,1.8,5.6,8.2,11.2,19.5,14.8,23.3,...,13.4,17.6,19.4,11.8,10.3,2.4,1.6,43887.0,57361.0,51.5
82,92154,22634.0,3.1,3.8,6.6,8.0,13.6,18.2,14.6,19.4,...,7.5,24.4,18.5,9.3,9.8,2.3,0.9,43831.0,55229.0,33.8
83,92173,8160.0,7.0,11.1,10.5,8.2,14.6,16.6,12.2,12.1,...,4.7,4.5,6.5,3.3,4.3,3.4,0.0,12992.0,29480.0,49.3


In [5]:

# Make new df with San Diego zipcodes
zipcodesdf = housing_data[["Geographic Area Name"]]
zipcodesdf

,Geographic Area Name
0,91901
1,91902
2,91906
3,91910
4,91911
...,...
80,92131
81,92139
82,92154
83,92173


In [8]:
zipcodesdf.rename(columns={'Geographic Area Name': 'Zip Code'}, inplace = True)
zipcodesdf

/var/folders/16/tnbbfsrj49l5pz272sb2fgt40000gn/T/ipykernel_82456/436232747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zipcodesdf.rename(columns={'Geographic Area Name': 'Zip Code'}, inplace = True)


,Zip Code
0,91901
1,91902
2,91906
3,91910
4,91911
...,...
80,92131
81,92139
82,92154
83,92173


In [6]:
# Read in US zipcode database
us_zip = pd.read_csv("Resources/RAW/uszips.csv")

In [7]:
# Create dataframe for just zipcodes and lat/lon
lat_lng = us_zip[['zip', 'lat', 'lng']]
# Rename the zip column to zipcode so we can merge
lat_lng.rename(columns={'zip':'Zip Code'}, inplace=True)
lat_lng.head()

/var/folders/16/tnbbfsrj49l5pz272sb2fgt40000gn/T/ipykernel_82456/3011836463.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lat_lng.rename(columns={'zip':'Zip Code'}, inplace=True)


,Zip Code,lat,lng
0,601,18.18027,-66.75266
1,602,18.36075,-67.17541
2,603,18.45744,-67.12225
3,606,18.16585,-66.93716
4,610,18.29110,-67.12243


In [9]:
# Merge dataframes on the zipcode
zip_lat = zipcodesdf.merge(lat_lng, on='Zip Code')
zip_lat.head(20)

,Zip Code,lat,lng
0,91901,32.81467,-116.71929
1,91902,32.67479,-117.00475
2,91906,32.66241,-116.47255
3,91910,32.63653,-117.06301
4,91911,32.60686,-117.04984
5,91913,32.62174,-116.98609
6,91914,32.66604,-116.95426
7,91915,32.62266,-116.95013
8,91932,32.56923,-117.11798
9,91935,32.70214,-116.78721


In [10]:
# Bring the index backto zipcode so we can iterate through safely.
zip_lat.set_index(['Zip Code'], inplace=True)

In [12]:
# Setting our API parameters
params = {
    "radius": 5000,
    "type": 'park',
    "key": API_KEY
}

In [13]:
# Iterate the responses and filter out market names in a zipcode.
park_list = []

for index, row in zip_lat.iterrows():
    lat = zip_lat.loc[index, 'lat']
    lng = zip_lat.loc[index, 'lng']
    
    
    params["location"] = f"{lat},{lng}"
    
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    parks = requests.get(base_url,params=params).json()

    park_list.extend(parks.get('results'))


In [14]:
# Confirm we got results
len(park_list)

1494

In [18]:
# Confirm we got coordinates for each market in a zipcode
park_list[0]


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 32.8194935, 'lng': -116.763489},
  'viewport': {'northeast': {'lat': 32.82063590000001, 'lng': -116.7614013},
   'southwest': {'lat': 32.8160663, 'lng': -116.7641849}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/park-71.png',
 'icon_background_color': '#4DB546',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/tree_pinlet',
 'name': "Wright's Field",
 'opening_hours': {'open_now': True},
 'photos': [{'height': 3024,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102936517431979644747">paterjo</a>'],
   'photo_reference': 'AcYSjRhk_2fy0r8GWgG6KnLYobB4cu5gvMrVnDBQjW-9fZlS4eQb0PSEcqMud2Odn5pQczA_YSvCcQ-U8rbbZTGTYTEcwg8UC8kTj0DN9IhgkDPYfRKPqd1cx7AB3jmllak2bGeLlk5X9tSJpg2Lz0kauhG48JrZL86DtqjmT7R69Na0_Tvu',
   'width': 4032}],
 'place_id': 'ChIJ19iiEcRg2YARKN4_P-d2D7I',
 'plus_code': {'compound_code': 'R69P+QJ Alpine, CA, USA',
  'global_code':

In [23]:
# Pull business operation status from our JSON
df2 = pd.DataFrame([x['business_status'] for x in park_list])
df2.set_axis(['business_statuss'], axis='columns', inplace=True)
df2

KeyError: 'business_status'

In [20]:
# Pull market names from our JSON
df3 = pd.DataFrame([x['name'] for x in park_list])
df3.set_axis(['name'],axis='columns', inplace=True)
df3

,name
0,Wright's Field
1,Viejas Park
2,Loveland Fishing Access
3,Sweetwater Summit Regional Park
4,Sweetwater County Park
...,...
1489,5th Marine Regiment Memorial
1490,Adventures trail
1491,San Onofre
1492,Trestles Beach Trailhead


In [21]:
# Pull lat/lon from our JSON
df4 = pd.DataFrame([x['geometry']['location'] for x in park_list])
df4

,lat,lng
0,32.819494,-116.763489
1,32.842437,-116.704128
2,32.799750,-116.760434
3,32.682508,-117.002547
4,32.682592,-117.001904
...,...,...
1489,33.422620,-117.556345
1490,33.343913,-117.515095
1491,33.389572,-117.593152
1492,33.396085,-117.591365


In [22]:
# Merge all our dataframes into one
final_park_df = pd.concat([df3, df4], axis = 1)
final_park_df

,name,lat,lng
0,Wright's Field,32.819494,-116.763489
1,Viejas Park,32.842437,-116.704128
2,Loveland Fishing Access,32.799750,-116.760434
3,Sweetwater Summit Regional Park,32.682508,-117.002547
4,Sweetwater County Park,32.682592,-117.001904
...,...,...,...
1489,5th Marine Regiment Memorial,33.422620,-117.556345
1490,Adventures trail,33.343913,-117.515095
1491,San Onofre,33.389572,-117.593152
1492,Trestles Beach Trailhead,33.396085,-117.591365


In [24]:
# Getting zipcodes for each market
import geopy

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']


geolocator = geopy.Nominatim(user_agent='user-agent')

zipcodes = final_park_df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='lat', lon_field='lng')


In [25]:
# Export dataframe as js file 
output_file = 'parks.js'
zipcodes.to_json(output_file, orient='records')

In [26]:
# Reading the js file and exporting to dataframe
zipsjs = pd.read_json('./parks.js')
zipsjs = pd.DataFrame(zipsjs)
zipsjs

,road,county,state,ISO3166-2-lvl4,postcode,country,country_code,amenity,house_number,tourism,...,commercial,industrial,subdivision,office,club,landuse,city_district,residential,natural,railway
0,Service Road,San Diego County,California,US-CA,91901.0,United States,us,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Willows Road,San Diego County,California,US-CA,91901.0,United States,us,Viejas Casino and Resort,5000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Japatul Road,San Diego County,California,US-CA,91901.0,United States,us,NaN,17741,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Summit Meadow Road,San Diego County,California,US-CA,91902.0,United States,us,NaN,NaN,SweetwaterCampground,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Summit Meadow Road,San Diego County,California,US-CA,91902.0,United States,us,NaN,NaN,SweetwaterCampground,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1489,NaN,San Diego County,California,US-CA,NaN,United States,us,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1490,Old Pacific Highway,San Diego County,California,US-CA,NaN,United States,us,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1491,Trestles Beach Trail,San Diego County,California,US-CA,92763.0,United States,us,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1492,Trestles Beach Trail,San Diego County,California,US-CA,92763.0,United States,us,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Checking the columns of the new dataframe
zipsjs.columns

Index(['road', 'county', 'state', 'ISO3166-2-lvl4', 'postcode', 'country',
       'country_code', 'amenity', 'house_number', 'tourism', 'shop', 'hamlet',
       'city', 'village', 'leisure', 'neighbourhood', 'place',
       'state_district', 'military', 'town', 'suburb', 'historic', 'retail',
       'building', 'man_made', 'highway', 'emergency', 'district', 'quarter',
       'commercial', 'industrial', 'subdivision', 'office', 'club', 'landuse',
       'city_district', 'residential', 'natural', 'railway'],
      dtype='object')

In [35]:
# Keeping relevant columns
parks_zipcodes = zipsjs[['postcode', 'county']]
parks_zipcodes = pd.DataFrame(parks_zipcodes)

In [36]:
# Merge all our dataframes into one
final_parks_df = pd.concat([df3, df4, parks_zipcodes], axis = 1)
final_parks_df


,name,lat,lng,postcode,county
0,Wright's Field,32.819494,-116.763489,91901.0,San Diego County
1,Viejas Park,32.842437,-116.704128,91901.0,San Diego County
2,Loveland Fishing Access,32.799750,-116.760434,91901.0,San Diego County
3,Sweetwater Summit Regional Park,32.682508,-117.002547,91902.0,San Diego County
4,Sweetwater County Park,32.682592,-117.001904,91902.0,San Diego County
...,...,...,...,...,...
1489,5th Marine Regiment Memorial,33.422620,-117.556345,NaN,San Diego County
1490,Adventures trail,33.343913,-117.515095,NaN,San Diego County
1491,San Onofre,33.389572,-117.593152,92763.0,San Diego County
1492,Trestles Beach Trailhead,33.396085,-117.591365,92763.0,San Diego County


In [37]:
# Removing NaN or other non-numeric values from the postcode column
final_parks_df = final_parks_df[pd.to_numeric(final_parks_df['postcode'], errors='coerce').notnull()]
final_parks_df


,name,lat,lng,postcode,county
0,Wright's Field,32.819494,-116.763489,91901.0,San Diego County
1,Viejas Park,32.842437,-116.704128,91901.0,San Diego County
2,Loveland Fishing Access,32.799750,-116.760434,91901.0,San Diego County
3,Sweetwater Summit Regional Park,32.682508,-117.002547,91902.0,San Diego County
4,Sweetwater County Park,32.682592,-117.001904,91902.0,San Diego County
...,...,...,...,...,...
1480,Trestles Beach Trailhead,33.395758,-117.591568,92763.0,San Diego County
1481,Rock Formation,33.402802,-117.551148,92051.0,San Diego County
1491,San Onofre,33.389572,-117.593152,92763.0,San Diego County
1492,Trestles Beach Trailhead,33.396085,-117.591365,92763.0,San Diego County


In [38]:
final_parks_df.postcode = final_parks_df.postcode.apply(int)
final_parks_df

/var/folders/16/tnbbfsrj49l5pz272sb2fgt40000gn/T/ipykernel_82456/845688103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_parks_df.postcode = final_parks_df.postcode.apply(int)


,name,lat,lng,postcode,county
0,Wright's Field,32.819494,-116.763489,91901,San Diego County
1,Viejas Park,32.842437,-116.704128,91901,San Diego County
2,Loveland Fishing Access,32.799750,-116.760434,91901,San Diego County
3,Sweetwater Summit Regional Park,32.682508,-117.002547,91902,San Diego County
4,Sweetwater County Park,32.682592,-117.001904,91902,San Diego County
...,...,...,...,...,...
1480,Trestles Beach Trailhead,33.395758,-117.591568,92763,San Diego County
1481,Rock Formation,33.402802,-117.551148,92051,San Diego County
1491,San Onofre,33.389572,-117.593152,92763,San Diego County
1492,Trestles Beach Trailhead,33.396085,-117.591365,92763,San Diego County


In [39]:
final_parks_df = final_parks_df[final_parks_df['county'] == 'San Diego County']
final_parks_df

,name,lat,lng,postcode,county
0,Wright's Field,32.819494,-116.763489,91901,San Diego County
1,Viejas Park,32.842437,-116.704128,91901,San Diego County
2,Loveland Fishing Access,32.799750,-116.760434,91901,San Diego County
3,Sweetwater Summit Regional Park,32.682508,-117.002547,91902,San Diego County
4,Sweetwater County Park,32.682592,-117.001904,91902,San Diego County
...,...,...,...,...,...
1480,Trestles Beach Trailhead,33.395758,-117.591568,92763,San Diego County
1481,Rock Formation,33.402802,-117.551148,92051,San Diego County
1491,San Onofre,33.389572,-117.593152,92763,San Diego County
1492,Trestles Beach Trailhead,33.396085,-117.591365,92763,San Diego County


In [40]:
# exporting file
from pathlib import Path

filepath = Path("Resources/Clean/San_Diego_Parks.csv")
final_parks_df.to_csv(filepath)